# 3 - Añadiendo memoria a la cadena

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/langchain.jpeg" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Cadena-con-memoria" data-toc-modified-id="1---Cadena-con-memoria-1">1 - Cadena con memoria</a></span></li></ul></div>

## 1 - Cadena con memoria

Primero necesitamos varios componentes necesarios para crear el chatbot:

+ `itemgetter`: Extrae elementos de un diccionario o lista.

+ `ChatOpenAI`: Un modelo de lenguaje proporcionado por OpenAI para manejar las interacciones de chat.

+ `ConversationBufferMemory`: Un componente para almacenar el historial de la conversación.

+ `RunnablePassthrough` y `RunnableLambda`: Son componentes para crear pasos de procesamiento en una cadena (pipeline).

+ `ChatPromptTemplate` y `MessagesPlaceholder`: Manejan la estructura del prompt que se enviará al modelo de lenguaje.

+ `StrOutputParser`: Transforma la salida de la cadena

In [1]:
# librerias 

from operator import itemgetter

from langchain_openai import ChatOpenAI

from langchain.memory import ConversationBufferMemory

from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.schema import StrOutputParser

In [2]:
# cargamos la API KEY de OpenAI

from dotenv import load_dotenv 
import os

# carga de variables de entorno
load_dotenv()


# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
# iniciamos el modelo de OpenAI

modelo = ChatOpenAI()

In [4]:
# definimos el prompt

prompt = ChatPromptTemplate.from_messages([('system', 'Eres un asistente.'),
                                           
                                           MessagesPlaceholder(variable_name='history'),
                                           
                                           ('human', '{pregunta}')
                                          ])

In [5]:
# parser de salida, transforma la salida a string

parser = StrOutputParser()

In [6]:
# definimos el objeto de memoria

memoria = ConversationBufferMemory(return_messages=True)

/var/folders/sq/8cd72b4n08vdtd2xm8zrhvm80000gq/T/ipykernel_18101/979582630.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memoria = ConversationBufferMemory(return_messages=True)


In [7]:
# iniciamos la memoria

memoria.load_memory_variables({})

{'history': []}

Ahora creamos la cadena o pipeline que procesa las entradas y las envía al modelo:

+ `RunnablePassthrough`: Define un paso que pasará las variables sin modificarlas.

    + `assign`: Se utiliza para asignar valores a las variables que se utilizan en la cadena. En este caso, asigna el historial, cargado desde la memoria mediante RunnableLambda.
    
    + `RunnableLambda`: Es utilizado para crear pasos personalizados dentro de una cadena o flujo de procesamiento. Este componente permite definir una función o lógica personalizada, usualmente una función lambda, que se puede ejecutar como parte de una secuencia más amplia de operaciones dentro de una cadena de LangChain.
    
    + `itemgetter('history')`: Extrae específicamente la variable "history" del resultado de load_memory_variables, un diccionario.

+ `| prompt`: Pasa la entrada, junto con el historial, al prompt definido anteriormente.

+ `| modelo`: El resultado del prompt se envía al modelo de lenguaje para generar una respuesta.

+ `| parser`: parser de salida, transforma la salida a string

In [8]:
# definimos la cadena

cadena = (RunnablePassthrough.assign(
          history=RunnableLambda(memoria.load_memory_variables) | itemgetter('history'))
         
          | prompt
         
          | modelo
          
          | parser
         )


In [9]:
# respuesta de la cadena

pregunta = {'pregunta': 'Hola, soy Pepe'}

respuesta = cadena.invoke(pregunta)

respuesta

'Hola, Pepe. ¿En qué puedo ayudarte hoy?'

In [10]:
# guarda la conversacion en la memoria

memoria.save_context(pregunta, {'respuesta': respuesta})

In [11]:
# memoria de la cadena

memoria.load_memory_variables({})

{'history': [HumanMessage(content='Hola, soy Pepe', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hola, Pepe. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})]}

In [12]:
# segunda respuesta de la cadena

pregunta = {'pregunta': '¿como me llamo?'}

respuesta = cadena.invoke(pregunta)

respuesta

'Te llamas Pepe. ¿Hay algo más con lo que pueda ayudarte?'